In [30]:
import pandas as pd
import io
import requests

url = "https://docs.google.com/spreadsheets/d/e/2PACX-1vTSLyp3Ev2bxe8pnHStFtbdwg-EhvbMEhEUzRsaFjC0sKVQ-OzyD8LbAjM9iYoG2LSmSTAGGFfed4I9/pub?gid=0&single=true&output=csv"

s=requests.get(url).content
df=pd.read_csv(io.StringIO(s.decode('utf-8')))
df.head()

,Level,Title,Text,Source,Type
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading


In [31]:
#clean data
import re
def clean(text):
    text = re.sub("Stadt X\,?\.?\s?(den )?\d+((\,)|(\.)|(\-))\d+((\,)|(\.)|(\. )|(\-))\d+ ", "", text)
    text = re.sub("^.{10,250}Sehr gee?h?e?e?rte?( Damen )?((und)|(oder))? He?er?ren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,100}Sehr Geherte mein Damen und Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,100}Liebe Dana", "Liebe Dana", text)
    text = re.sub("^.{10,100}Lieber? Silke", "Liebe Silke", text)
    text = re.sub("^.{10,100}Liebe Maria", "Liebe Maria", text)
    text = re.sub("^.{10,100}Liebe Katharina", "Liebe Katharina", text)
    text = re.sub("(\S+)\.(\S+)", r"\1. \2", text)
    text = re.sub("www. ", r"www.", text)
    text = re.sub("^.{10,100}Lieber Michael", "Lieber Michael", text)
    text = re.sub("^.{10,100}Lieber Ingo", "Lieber Ingo", text)
    text = re.sub("^.{10,100}Lieber Julia", "Lieber Julia", text)
    text = re.sub("^.{10,100}Lieber Eva", "Lieber Eva", text)
    text = re.sub("^.{10,100}Hallo Eva", "Hallo Eva", text)
    text = re.sub("^.{10,100}Liebe Julia", "Liebe Julia", text)
    text = re.sub("^.{10,100}Hallo Julia", "Hallo Julia", text)
    text = re.sub("^.{10,200}Sehr geerhte Damen und Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Sehr Geehrte Damen und Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Sehr geehrte Damen und Herrn", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}See?hr gerrhte Herren und Damen", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Sehr geehte Damen und Herr", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Seehr geehrte Damen und Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Seher geherte Damen und Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Sehr geherte Herrn", "Sehr geehrte Herren", text)
    text = re.sub("^.{10,200}Sehr geehrte Fir", "Sehr geehrte Fir", text)
    text = re.sub("^.{10,200}Sehr geehrte Damen u. Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Sehr geehrte Damen u Herren", "Sehr geehrte Damen und Herren", text)
    text = re.sub("^.{10,200}Seehr Geehrte Damen und He", "Sehr geehrte Damen und He", text)
    text = re.sub("^.{10,200}Sehr geehrte Au-", "Sehr geehrte Au-", text)
    return text
    


In [32]:
df.head()

,Level,Title,Text,Source,Type
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading


In [5]:
import urllib
url = "http://www.cis.uni-muenchen.de/~schmid/tools/RFTagger/data/RFTagger.tar.gz"
filename = "RFTagger.tar.gz"
urllib.request.urlretrieve(url, filename)
import tarfile
tar = tarfile.open(filename)
tar.extractall()
tar.close()

In [6]:
from subprocess import run, check_output

def preprocessing(text):
    file = open("RFTagger/temp.txt", "w")
    file.write(text)
    file.close()
    print("RFTagger", text[:20])
    test_tagged = check_output(["cmd/rftagger-german", "temp.txt"], cwd="RFTagger").decode("utf-8").split("\n")

    sent_indecies = [index for index in range(len(test_tagged)) if test_tagged[index] == '']
    sent_indecies = [-1] + sent_indecies

    test_tagged_saparated= []
    for entry in test_tagged:
        temp = entry.split("\t")
        if len(temp) == 3:
            test_tagged_saparated.append([temp[0]] + temp[1].split(".") + [temp[2]])
        else:
            test_tagged_saparated.append([])

    output = []
    for start, end in zip(sent_indecies, sent_indecies[1:]):
        output.append(test_tagged_saparated[start+1: end])

    return output

In [33]:
#stanford parser
from stanfordcorenlp import StanfordCoreNLP
from nltk.tokenize import sent_tokenize
import re
import sys

    
def parsing(text):
    output=[]
    text = sent_tokenize(text)
    trees=[]
    
    for sentence in text:
        print("Stanford Parser", sentence)
        try:
            tree = re.sub(" +", " ", nlp.parse(sentence).replace("\n", ""))
            output.append(tree)
        except ValueError:
            output.append("")
        
    return output

In [34]:
df["cleanedText"] = [clean(text) for text in df["Text"]]
#df["preprocessedText"] = [preprocessing(text) for text in df["cleanedText"]]
nlp = StanfordCoreNLP(r'/home/soeren/stanford-corenlp-full-2018-02-27', lang='de', memory='4g')
df["parsedText"] = [parsing(text) for text in df["cleanedText"]]
nlp.close() # Do not forget to close! The backend server will consume a lot memory.

Stanford Parser Zu meiner Familie gehören vier Personen.
Stanford Parser Die Mutter bin ich und dann gehört natürlich mein Mann dazu.
Stanford Parser Wir haben zwei Kinder, einen Sohn, der sechs Jahre alt ist und eine dreijährige Tochter.
Stanford Parser Wir wohnen in einem kleinen Haus mit einem Garten.
Stanford Parser Dort können die Kinder ein bisschen spielen.
Stanford Parser Unser Sohn kommt bald in die Schule, unsere Tochter geht noch eine Zeit lang in den Kindergarten.
Stanford Parser Meine Kinder sind am Nachmittag zu Hause.
Stanford Parser So arbeite ich nur halbtags.
Stanford Parser Eigentlich gehören zu unserer Familie auch noch die Großeltern.
Stanford Parser Sie wohnen nicht bei uns.
Stanford Parser Sie haben ein Haus in der Nähe.
Stanford Parser Die Kinder gehen sie oft besuchen.
Stanford Parser Mein Name ist Maria.
Stanford Parser Ich bin 30 Jahre alt.
Stanford Parser Zu meiner Familie gehören außer mir noch drei weitere Personen.
Stanford Parser Mein Mann Markus ist 31 

Stanford Parser Junge Hunde wollen noch viel unternehmen und so geht Jan jeden Tag mit ihm spazieren.
Stanford Parser Struppi hat immer Hunger.
Stanford Parser Er bekommt morgens seine Hundekuchen, aber mittags will er schon wieder etwas essen.
Stanford Parser Jan gibt ihm aber nichts.
Stanford Parser Erst am Abend bekommt Struppi noch eine Mahlzeit.
Stanford Parser Struppi schläft in einem runden Korb.
Stanford Parser Dort fühlt er sich sehr wohl.
Stanford Parser Wenn Jan in der Schule ist, ist Struppi traurig und wartet auf ihn.
Stanford Parser Er freut sich sehr, wenn sein Freund wieder zurück nach Hause kommt.
Stanford Parser Jan möchte nie mehr ohne Hund leben.
Stanford Parser Heute ist der erste Schultag.
Stanford Parser Lena steht mit ihrer Schultüte vor der Schule.
Stanford Parser Sandra, Susanne und Paul sind auch da.
Stanford Parser Die Kinder kennen sich aus dem Kindergarten.
Stanford Parser Jetzt gehen sie in die gleiche Klasse.
Stanford Parser Sie freuen sich schon auf den

Stanford Parser Frau Meier und Frau Schmidt gehen zusammen in den Supermarkt.
Stanford Parser Frau Meier: “Ich brauche Tomaten.
Stanford Parser Mein Mann will Salat essen.
Stanford Parser Ich nehme zehn Tomaten.
Stanford Parser "Frau Schmidt: “Tomaten sind gut.
Stanford Parser Ich kaufe fünf Tomaten.
Stanford Parser Da ist auch Kopfsalat. "
Stanford Parser Frau Meier nimmt keinen Kopfsalat.
Stanford Parser Sie nimmt aber zwei Gurken.
Stanford Parser Frau Meier kauft auch ein Kilo Zwiebeln.
Stanford Parser Frau Schmidt will Brot kaufen.
Stanford Parser Frau Meier: “Das Brot ist nicht schön.
Stanford Parser Ich kaufe Brot in der Bäckerei.
Stanford Parser Aber die Schokolade hier ist gut.
Stanford Parser Sie kostet nur 50 Cent.
Stanford Parser Ich nehme drei Tafeln Schokolade.
Stanford Parser "Frau Schmidt: “Da ist noch ein Sonderangebot.
Stanford Parser Mineralwasser und Orangensaft sind billig. "
Stanford Parser Frau Meier: “Orangensaft habe ich zu Hause.
Stanford Parser Aber ich brauch

Stanford Parser Seine Großmutter feiert heute einen runden Geburtstag*, sie wird 70 Jahre alt.
Stanford Parser Seit Wochen sprechen seine Eltern nur noch über diesen Tag.
Stanford Parser Es wird ein großes Familienfest sein, in einem Saal, welcher über viele Tische und Stühle verfügt.
Stanford Parser Großmama liebt die Familie und freut sich jedes Mal, wenn alle da sind.
Stanford Parser 5 Kinder, 10 Enkelkinder und 3 Urenkelkinder**, dazu ihre beiden Geschwister und natürlich Großvater.
Stanford Parser Paul kann sich nicht mehr an den letzten runden Geburtstag seiner Großmutter erinnern, da er damals erst 2 Jahre alt war.
Stanford Parser Nun ist er 12 Jahre und seine Schwester und sein Bruder kleiden sich festlich.
Stanford Parser Paul trägt zur Feier des Tages ein weißes Hemd mit einer schwarzen Krawatte.
Stanford Parser Papa und Mama sind sehr in Eile und wollen pünktlich im Festsaal sein.
Stanford Parser Was es wohl zu Essen gibt?
Stanford Parser Oma liebt Rinderbraten über alles un

Stanford Parser Tina: „Nein, ich bin Studentin und habe heute keine Vorlesungen.
Stanford Parser Deswegen habe ich auch Zeit. "
Stanford Parser Alex: „Aha, das ist ja interessant.
Stanford Parser Ich bin auch Student und studiere Physik.
Stanford Parser Was studieren Sie denn?
Stanford Parser Tina: „Medizin, genauer gesagt Tiermedizin.
Stanford Parser Ich möchte mal Tierärztin werden. "
Stanford Parser Alex: „Das ist ja klasse.
Stanford Parser Ich habe selber auch zwei Haustiere.
Stanford Parser Eine Katze und einen Goldfisch.
Stanford Parser Wenn die mal krank sind, weiß ich ja jetzt, wo ich hingehen muss. "
Stanford Parser Tina: „Na ja, soweit bin ich noch nicht.
Stanford Parser Aber es würde mich freuen Ihre Haustiere mal kennenzulernen. "
Stanford Parser Alex: „Sehr gerne, sobald ich eingezogen bin und ein wenig aufgeräumt habe, sind Sie herzlich eingeladen. "
Stanford Parser Tina: „Vielen Dank, ich werde vorbeischauen.
Stanford Parser Bis bald und viel Erfolg beim Einzug. "
Stanfo

...

Stanford Parser Meiner Meinung nach, Deutschlernen ist für mich sehr wichtig, weil die Tourismus zwischen der Türkei und Deutschland sehr entwickelt ist.
Stanford Parser Die Gesellschaften der jeweiligen Länder müssen miteinander gut verständigen.
Stanford Parser Nicht nur der Tourismus sondern auch der In- und Export zwischen den beiden Ländern ist groß.
Stanford Parser Auserdem die Deutschesprache finde ich sehr interessand, weil auch in vielen europeäischen Ländern zum Beispiel in Österreich und Schweiz die Menschen Deutsch sprechen.
Stanford Parser Ich bin eine ehemalige Türkisch-Lehrerin und wenn ich die Deutschesprache auch gut lernen und Niveau C1 Zertifikat bestehen kann, dann kann ich in meinen erlernten Beruf zurückkehren und hier in Deutschland weiter ausüben.
Stanford Parser Ich bin der Überzeugung, dass ich mit meinen deutschen und türkischen Kenntnissen den ausländischen Schüllern sehr gut behilflich sein kann.
Stanford Parser Aus diesen Gründen, Deutschlernen ist für mic

Stanford Parser Aber Nützlichkeit ist nicht das einzige Kriterium für Fremdsprachenlernen.
Stanford Parser Vor allem Hessen hat so viel anzubieten: historische Städte, moderne Einkaufszonen, romantische Parke.
Stanford Parser Das Land motiviert viele Ausländer die Sprache dieses Landes zu lernen um das Land besser verstehen.
Stanford Parser Manche kommen für ein Paar Monate und bleiben bis Ende ihres Lebens.
Stanford Parser Die obengenannten Gründe überzeugen mich, dass es sinnvoll ist, Deutsch zu lernen.
Stanford Parser Die Kultur ist ein wichtiger Teil uneres Lebens.
Stanford Parser Da Deutschland ein multikultureles Land ist, sollen die Menschen verschiedener Nationen zusammen leben.
Stanford Parser Is es sinvoll, auch im Ausland die Traditionen seines Heimatlandes fortzusetzen, oder sollte man sich an der Kultur des Gastlandes orientieren?
Stanford Parser Es ist nicht einfach, auf diese Frage zu antworten.
Stanford Parser Einerseits sollen die Migranten sich integrieren, damit sie 

Stanford Parser Wie die Vielfalt von Menschen und verschiedenen Kulturen, so die Vielfalt unterschiedlicher Sprachen machen das Leben auf der Erde spannender, interessanter, bunter.
Stanford Parser Deutschlernen muss sein!
Stanford Parser Ob man die deutsche Sprache lernen muss?
Stanford Parser Natürlich werde ich diese Frage bejaen.
Stanford Parser Man sagt, wir leben in der Zeit, in der Englisch, Französisch und Spanisch immer mehr Popularität erwerben und „nützlicher" angesehen werden.
Stanford Parser Das klingt untröstlich, heißt aber nicht, dass die deutsche Sprache nicht erlernt werden muss.
Stanford Parser Es wäre zu Schade, die Werke von Goethe und Heine im Original nicht lesen zu können.
Stanford Parser Die Geschichte und Kultur mit der eigenen Sprache zu erleben ist wirklich verlockend.
Stanford Parser Das Gefuhl, dass ich deutsch sprechen und lesen kann, erfüllt mich mit der Freude.
Stanford Parser Ich kann die Deutschen besser verstehen, ihr Lebensstil, Gewohheiten.
Stanfor

Stanford Parser Die Situation führt dazu, dass die Einheimischen die Menschen nicht sympathisch finden und willen mit ihnen kein Kontakt knüpfen.
Stanford Parser Da bin ich sicher, dass diese Situation mit der Hilfe der Bildung gelöst werden kann, aber für die Veränderung beide Gruppen arbeiten müssen.
Stanford Parser (Was denkt ihr darüber?)
Stanford Parser Eva
Stanford Parser Assimilation?
Stanford Parser Heute begegnet man immer mehr Gastarbeiter oder Austauschschüler in aller Länder der Welt.
Stanford Parser Ob diese Leute sich an der Kultur des Gastlandes orientieren sollen, liegt, meiner Meinung nach, an die Zeit, die sie im Ausland verbringen.
Stanford Parser Wenn man schon am Anfang seiner Anwesenheit weiß, dass er mehr als zwei Jahre im Ausland verbringen will, soll er sich schon vorbereiten, an die Traditionen des gewählten Landes sich zu gewöhnen.
Stanford Parser Um Freunde machen zu können, um in einer Restaurant bestellen zu können, oder ganz einfach auf der Straße spazier

Stanford Parser In den letzten Zeiten spricht man immer mehr über das Problem von Leuten, die aus ihrem ursprünglichen Heimat flüchten.
Stanford Parser Diese Personen sind als "Migranten" bezeichnet.
Stanford Parser In den meisten Fällen landen diese Personen nach einer langwierigen und abenteuerlichen Flucht in unser Land.
Stanford Parser Die Kulturen aus den sie stammen sind diversen, und auch die Einstellungen und Meinungen der Leuten gegenüber die Migranten sehr unterschiedlich sind.
Stanford Parser Ferner sind die Auswanderer oft ohne Dokumente und krank, und auf diesem Weg wird das Hilfe der Ärtzten und Freiwilligen nutzvoll.
Stanford Parser Nach diesen ersten Schritten, wenn die Migranten in den neuen Land sich integriert haben, entsteht das Risiko, dass sie ihre Traditionen und Kultur verlieren können.
Stanford Parser Wenn man im Ausland für eine Weile wohnt, ist es fast unmöglich unsere Traditionen nicht ändern, weil die neue Kultur und die neue "Welt" in dem wir wohnen eine u

In [35]:
display(df.head(n=100))
filename = "preprocessed_text_df.csv"
df.to_csv(filename)

,Level,Title,Text,Source,Type,cleanedText,parsedText
0,A1,Meine Familie,Zu meiner Familie gehören vier Personen. Die M...,https://german.net/reading/familie/,Reading,Zu meiner Familie gehören vier Personen. Die M...,[(ROOT (S (PP (APPR Zu) (PPOSAT meiner) (NN Fa...
1,A1,Maria und ihre Familie,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,https://german.net/reading/marias-familie/,Reading,Mein Name ist Maria. Ich bin 30 Jahre alt. Zu ...,[(ROOT (S (NP (PPOSAT Mein) (NN Name)) (VAFIN ...
2,A1,Ich bin Tom,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,https://german.net/reading/tom/,Reading,Hallo! Ich bin Tom Maier. Ich bin 13 Jahre alt...,"[(ROOT (NUR (ITJ Hallo) ($. !))), (ROOT (S (PP..."
3,A1,Freundinnen,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,https://german.net/reading/freundinnen/,Reading,Ricarda ist 21 Jahre alt und wohnt in Lübeck. ...,[(ROOT (CS (S (NE Ricarda) (VAFIN ist) (AP (NP...
4,A1,Einkaufen im Supermarkt,Frau Meier geht heute in den Supermarkt. Ihr M...,https://german.net/reading/einkaufen/,Reading,Frau Meier geht heute in den Supermarkt. Ihr M...,[(ROOT (S (NP (NN Frau) (NE Meier)) (VVFIN geh...
5,A1,Frühstück,Es ist 7 Uhr am Morgen. Familie Müller sitzt a...,https://german.net/reading/fruehstueck/,Reading,Es ist 7 Uhr am Morgen. Familie Müller sitzt a...,[(ROOT (S (PPER Es) (VAFIN ist) (NP (CARD 7) (...
6,A1,Susanne schreibt einen Brief,Lieber Thomas! Jetzt bist du weit weg! Ich bin...,https://german.net/reading/brief/,Reading,Lieber Thomas! Jetzt bist du weit weg! Ich bin...,[(ROOT (S (ADJD Lieber) (NP (NE Thomas)) ($. !...
7,A1,Der Skiurlaub,Hartmut hat im Februar eine Reise gemacht: Er ...,https://german.net/reading/skiurlaub/,Reading,Hartmut hat im Februar eine Reise gemacht: Er ...,[(ROOT (CS (S (NE Hartmut) (VAFIN hat) (VP (PP...
8,A1,Struppi,Jan hat einen kleinen Hund. Er heißt Struppi u...,https://german.net/reading/struppi/,Reading,Jan hat einen kleinen Hund. Er heißt Struppi u...,[(ROOT (S (NE Jan) (VAFIN hat) (NP (ART einen)...
9,A1,In der Schule,Heute ist der erste Schultag. Lena steht mit i...,https://german.net/reading/schule/,Reading,Heute ist der erste Schultag. Lena steht mit i...,[(ROOT (S (ADV Heute) (VAFIN ist) (NP (ART der...
